In [1]:
import json
import matplotlib.pyplot as plt
import datetime, time
import pytz
import pandas as pd

In [2]:
hashtag = '#SuperBowl'
hashtag_dict = {'#GoHawks' : 'tweets_#gohawks.txt',
                '#GoPatriots' : 'tweets_#gopatriots.txt',
                '#NFL' : 'tweets_#nfl.txt',
                '#Patriots' : 'tweets_#patriots.txt',
                '#SB49' : 'tweets_#sb49.txt',
                '#SuperBowl' : 'tweets_#superbowl.txt'}

input_file = open('./tweet_data/' + hashtag_dict[hashtag])
pst_tz = pytz.timezone('US/Pacific')
start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))
total_list = []
i = 0
for line in input_file:
    curr_list = []
    curr_data = json.loads(line)
    citation_date = curr_data['citation_date']
    
    location = curr_data['tweet']['user']['location']
    text = curr_data['highlight']
    curr_list.append(text)
    curr_list.append(location)
    if citation_date > end_time:
        total_list.append(curr_list)
        i += 1
    

In [3]:
Data = pd.DataFrame(total_list,columns=['text','location'])
Data = Data.reset_index(drop=True)
print(Data[:]['location'])

0                               MALAYSIA
1                                   Ohio
2                                       
3            #NinerEmpire  #GoLobos  #23
4                             Dallas, TX
5                                       
6                LA ~ Chicago ~ NYC ~ FL
7                                       
8                             Boston, MA
9                                       
10        Total Divas on E! & RAW on USA
11                                      
12                               Arizona
13                    Dare you to watch 
14                                      
15                                      
16                   Piro, Osa Peninsula
17                 Copenhagen & New York
18                                Boston
19                                      
20                             Vancouver
21                            Malibu, CA
22              United States of America
23                     Herndon, Virginia
24              

In [23]:
import re
from textblob import TextBlob
text_w = []
location_w = []
text_m = []
location_m = []

list_w = ['Washington', 'WA', 'washington', 'WASHINGTON']
list_m = ['Massachusetts', 'massachusetts', 'MA', 'MASSACHUSETTS' ]
list_dc = ['DC', 'D.C.', 'dc', 'd.c.']

i = 0
j = 0
for index, row in Data.iterrows():
    list_row1 = re.split('; |, ',row[1])
    if any(x in list_row1 for x in list_w) and not any(y in list_row1 for y in list_dc):
        text = row[0].split('http')
        temp = re.sub('\s#', ' ', text[0], flags=re.IGNORECASE)
        testimonial = TextBlob(temp)
        
        text_w.append(testimonial.sentiment.polarity)
        location_w.append(0)
        i += 1
    elif any(x in list_row1 for x in list_m):
        text = row[0].split('http')
        temp = re.sub('\s#', ' ', text[0], flags=re.IGNORECASE)
        testimonial = TextBlob(temp)
        if testimonial.sentiment.polarity < -0.2: 
            text_m.append(testimonial.sentiment.polarity)
            location_m.append(1)
            j += 1
        



In [24]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from textblob import TextBlob
from sklearn.svm import SVC

def plot(test_target, predicted):
    # Compute fpr, tpr, thresholds and roc auc
    fpr, tpr, thresholds = metrics.roc_curve(test_target, predicted)
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()
def performance(clf):
    scoring = ['accuracy', 'precision_macro', 'recall_macro']
    x = text_w + text_m
    import numpy as np
    x = np.asarray(x)
    x = np.repeat(x,2)
    x = np.reshape(x,(-1,2))
    scores = cross_validate(clf, x, location_w + location_m, scoring=scoring, cv=5, return_train_score=False)
    mean = scores['test_accuracy'].mean()
    recall = scores['test_recall_macro'].mean()
    precision = scores['test_precision_macro'].mean()


    print('accuracy:%s'%mean)
    
    print('recall:%s'%recall)
    print('precision:%s'%precision)
    
    
    

In [25]:
clf = SVC(probability=True, C = 1, random_state=42)
performance(clf)

accuracy:0.9018841800551611
recall:0.9320803842450701
precision:0.8293157977598848
